In [1]:
import json

import requests

from folium import Map, TileLayer, GeoJson

In [2]:
endpoint = "http://0.0.0.0:8082"

In [3]:
geojson = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -113.8623046875,
          31.316101383495624
        ],
        [
          -79.1455078125,
          31.316101383495624
        ],
        [
          -79.1455078125,
          49.15296965617039
        ],
        [
          -113.8623046875,
          49.15296965617039
        ],
        [
          -113.8623046875,
          31.316101383495624
        ]
      ]
    ]
  }
}
bounds = (-113.8623046875, 31.316101383495624, -79.1455078125, 49.15296965617039)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=4
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

### Get File info

In [16]:
# create Mosaic
body = {
    "collections": ["landsat-8-c2-l2"],
}

r = requests.post(
    f"{endpoint}/create",
    json=body,
    params={
        # Info to add to the tilejson response
        "minzoom": 8,
        "maxzoom": 12,
        "bounds": ",".join(map(str, bounds)),
        # query parameter to add to the tile URL
        "assets": "SR_B1",
        "rescale": "0,20000",
    }
).json()
print(r)

{'tilejson': '2.2.0', 'name': '1405d4e4ecb6b5a9aecf16d62d285906', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['http://0.0.0.0:8082/tiles/1405d4e4ecb6b5a9aecf16d62d285906/WebMercatorQuad/{z}/{x}/{y}@1x?assets=SR_B1&rescale=0%2C20000'], 'minzoom': 8, 'maxzoom': 12, 'bounds': [-113.8623046875, 31.316101383495624, -79.1455078125, 49.15296965617039], 'center': [-96.50390625, 40.23453551983301, 8]}


In [19]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=7
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    attr="Mosaic"
)
aod_layer.add_to(m)
m